In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torchvision.transforms as transforms
import torchvision.models as models

import copy, os, time, sys, h5py, pickle, argparse
from sklearn.cross_decomposition import PLSRegression
from tqdm import tqdm

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def get_model(layer_name = 'pool4'):
    cnn = models.vgg19(pretrained=True).features.to(device).eval()
    
    i=j=1
    model = nn.Sequential()
    layer_names = []
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            name = 'conv{}_{}'.format(i, j)
            j+=1
        elif isinstance(layer, nn.ReLU):
            name = 'relu{}_{}'.format(i, j)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool{}'.format(i)
            i+=1; j=1;
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))
        model.add_module(name, layer)
        if name == layer_name:
            break
    return model

def load_fit_weights(layer, area, ver='V6'):
    ROOTDIR = '/home/gru/akshay/neurint'
    if area == 'V1':
        filepath = f'{ROOTDIR}/model_fits/{area}_vgg19_{layer}-fit_5-components.pickle'
    else:
        filepath = f'{ROOTDIR}/model_fits/{area}_vgg19_{layer}-fit_5-components_{ver}.pickle'
    print(filepath)
    if os.path.isfile(filepath):
        results = np.load(filepath, allow_pickle=True)
        weights = np.array(results[f'{area.lower()}_weights']).squeeze()
        return weights
    else:
        print(f'Weight file not found : {filepath}')
        return None

def image_loader(image_name):
    imsize = 256 if torch.cuda.is_available() else 128  # use small size if no gpu

    loader = transforms.Compose([
            transforms.Resize(imsize),  # scale imported image
            transforms.CenterCrop(imsize),
            transforms.ToTensor()])  # transform it into a torch tensor

    image = Image.open(image_name)
    # fake batch dimension required to fit network's input dimensions
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

def get_neural_model_response(images, layer, area):
    model = get_model(layer)
    W = torch.from_numpy(load_fit_weights(layer, area).T).to(device)

    neural_responses = [] 
    for i_image in tqdm(range(len(images)), desc='Extracting model responses'): 
        input_image = image_loader(images[i_image])
        model_response = model(input_image).detach()
        neural_response = torch.matmul(model_response.float(), W.float()).detach().cpu().numpy()

        neural_responses.append(neural_response.flatten())
    return np.array(neural_responses)

cpu


In [21]:
images = ['outputs/bw/rocks_leaves_0_pool4-IT_s1.png', 
          'outputs/bw/rocks_leaves_25_pool4-IT_s1.png',
          'outputs/bw/rocks_leaves_50_pool4-IT_s1.png',
          'outputs/bw/rocks_leaves_75_pool4-IT_s1.png',
          'outputs/bw/rocks_leaves_100_pool4-IT_s1.png']

IT_responses = get_neural_model_response(images, 'pool4', 'IT')

/home/gru/akshay/neurint/model_fits/IT_vgg19_pool4-fit_5-components_V6.pickle
Weight file not found : /home/gru/akshay/neurint/model_fits/IT_vgg19_pool4-fit_5-components_V6.pickle


AttributeError: 'NoneType' object has no attribute 'T'